# 1. Request All Data via API
Before creating the dashboard, I must collect all of the data. This notebook outlines this process. While requesting all of the data can be time consuming due to rate limiting, it is useful for resetting the data if something goes wrong.

## 1.1 Load Libraries and Functions

In [1]:
library(tidyverse)
library(httr)
library(jsonlite)
library(anytime)
library(here)
library(stringi)
library(googlesheets4)

source(here("R", "00_source.R"))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: 'jsonlite'


The following object is masked from 'package:purrr':

    flatten


here() starts at C:/Users/Hunter



## 1.2 Get Track Info

In [2]:


APIKey <- "2eb9b95751e6a5cd1da827604bf51906"
userAgent <- "CACTUS__PLANT"
url <- "https://ws.audioscrobbler.com/2.0/"

In [3]:
pageNum <- 1
totalPages <- 2

scrobbleData <- data.frame(matrix(vector(), 0, 1,
                                  dimnames=list(c(), c(""))),
                           stringsAsFactors=F)

In [4]:
while(pageNum <= totalPages) {
    # request data
    payload <- list(method = "user.getRecentTracks", user = userAgent, extended = 1, limit = 200, page = pageNum)
    res <- get.lastfm(payload, userAgent)
  
    # error if request unsuccessful
    if (res$status_code != 200) {
        print(paste("ERROR:", res$status_code))
        break
    }
  
    # convert data to dataframe
    char <- rawToChar(res$content)
    dataJSON <- fromJSON(char)
    
    # save data in dataframe
    newTracks <- data.frame(artistName = dataJSON$recenttracks$track$artist$name,
                            artistURL = dataJSON$recenttracks$track$artist$url,
                            albumName = dataJSON$recenttracks$track$album$`#text`,
                            trackName = dataJSON$recenttracks$track$name,
                            trackURL = dataJSON$recenttracks$track$url,
                            trackImage = unlist(lapply(lapply(dataJSON$recenttracks$track$image, `[[`, 2), `[[`, 4)),
                            date = as.numeric(dataJSON$recenttracks$track$date$uts))
  
    # update pages
    totalPages <- as.numeric(dataJSON$recenttracks$`@attr`$totalPages)
    pageNum <- as.numeric(dataJSON$recenttracks$`@attr`$page)
  
    # add new data to dataframe
    scrobbleData <- rbind(scrobbleData, newTracks)
  
    Sys.sleep(0.25) # reduce frequency of requests to not overload server
  
    pageNum <- pageNum + 1
  
}

In [5]:
str(scrobbleData)

'data.frame':	16463 obs. of  7 variables:
 $ artistName: chr  "Alex G" "Underscores" "Zen Sounds" "Zen Sounds" ...
 $ artistURL : chr  "https://www.last.fm/music/Alex+G" "https://www.last.fm/music/Underscores" "https://www.last.fm/music/Zen+Sounds" "https://www.last.fm/music/Zen+Sounds" ...
 $ albumName : chr  "Rocket" "boneyard aka fearmonger" "Loopable White Noise for Yoga & Meditation, Vol. 1" "Loopable White Noise for Yoga & Meditation, Vol. 1" ...
 $ trackName : chr  "Judge" "Everybody's dead!" "Rain Sounds: Long Rainstorm (Loopable)" "Rain Sounds: Long Rainstorm (Loopable)" ...
 $ trackURL  : chr  "https://www.last.fm/music/Alex+G/_/Judge" "https://www.last.fm/music/Underscores/_/Everybody%27s+dead!" "https://www.last.fm/music/Zen+Sounds/_/Rain+Sounds:+Long+Rainstorm+(Loopable)" "https://www.last.fm/music/Zen+Sounds/_/Rain+Sounds:+Long+Rainstorm+(Loopable)" ...
 $ trackImage: chr  "https://lastfm.freetls.fastly.net/i/u/300x300/da1abef9f05307110b46969196768164.jpg" "https://lastfm

# 1.3 Get Tag Info

In [6]:
artists <- unique(scrobbleData$artistName)
artistTagInfo <- data.frame(artistName = artists,
                            artistTag = character(length(artists)))

In [7]:
for (i in 1:length(artists)) {
    # request data
    payload <- list(method = "artist.getTopTags", artist = artistTagInfo$artistName[i])
    res <- get.lastfm(payload, userAgent)
  
    # error if request is unsuccessful
    if (res$status_code != 200) {
        print(paste("ERROR:", res$status_code))
        break
      }
  
    # convert data
    char <- rawToChar(res$content)
    dataJSON <- fromJSON(char)
  
    tagName <- dataJSON$toptags$tag$name[1]
    
    # save data to dataframe
    if(!is.null(tagName)) {
        artistTagInfo$artistTag[i] <- tagName
    } else {
        artistTagInfo$artistTag[i] <- NA
    }
  
    Sys.sleep(0.25) # reduce frequency of requests to not overload server
  
}

## 1.3 Save Data to Google Sheet

In [8]:
scrobbleDataID <- "1-hx0pjREakkdFIQZWJJ5-KZbSQNTEdX99GQ0HM8u6MM"
artistTagID <- "1lZLASIwaiXjQ8JBQRwnIH9LLrqijM68hGKAyrOdfdgU"

In [10]:
write_sheet(scrobbleData, scrobbleDataID, 1)
write_sheet(artistTagInfo, artistTagID, 1)

✔ Writing to my_scrobble_data.

✔ Writing to sheet Sheet1.

✔ Writing to artist_tag_data.

✔ Writing to sheet Sheet1.

